In [11]:
import pandas as pd

# Read the Excel file
df = pd.read_excel (r"data.xlsx")

# Drop rows with any NaN values
df = df.dropna()

# Convert to CSV
df.to_csv("data.csv", index=False)

# Read the newly created CSV
df = pd.read_csv("data.csv")

# Display the first 10 rows
df.head(10)




,UserID,Activity,TimeOfDay,Season,ExternalTemp,RoomTemp,RoomHumidity,FlowRate,ColdWaterTemp,DesiredTemp
0,1,Shower,Morning,Winter,5,20,45,9,10,42
1,2,Hand Washing,Evening,Summer,30,24,55,5,25,35
2,3,Dishwashing,Afternoon,Spring,20,22,50,12,15,45
3,4,Laundry,Morning,Autumn,10,19,40,15,12,30
4,5,Shower,Evening,Winter,0,18,50,10,5,40
5,6,Hand Washing,Afternoon,Spring,22,21,45,6,20,33
6,7,Dishwashing,Morning,Summer,28,26,60,11,23,38
7,8,Laundry,Evening,Winter,-5,17,55,14,8,28
8,9,Shower,Afternoon,Autumn,15,22,48,9,16,39
9,10,Hand Washing,Morning,Summer,35,25,50,5,30,30


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   UserID         299 non-null    int64  
 1   Activity       299 non-null    object 
 2   TimeOfDay      299 non-null    object 
 3   Season         299 non-null    object 
 4   ExternalTemp   299 non-null    int64  
 5   RoomTemp       299 non-null    int64  
 6   RoomHumidity   299 non-null    int64  
 7   FlowRate       299 non-null    float64
 8   ColdWaterTemp  299 non-null    float64
 9   DesiredTemp    299 non-null    float64
dtypes: float64(3), int64(4), object(3)
memory usage: 23.5+ KB


In [12]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Encode categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(df[['Activity', 'TimeOfDay', 'Season']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Activity', 'TimeOfDay', 'Season']))

# Concatenate encoded features with the rest of the data
df_encoded = pd.concat([df.drop(['Activity', 'TimeOfDay', 'Season'], axis=1), encoded_df], axis=1)

# Define features and target variable
X = df_encoded.drop(['UserID', 'DesiredTemp'], axis=1)
y = df_encoded['DesiredTemp']



# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Assuming X_train and y_train are your feature matrix and target vector
X_train = X_train.dropna()
y_train = y_train[X_train.index]  # Align y_train with the updated X_train

In [13]:
import joblib

# Initialize and train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R2 Score: {r2}')


# Example new data for prediction
new_data = {
    'ExternalTemp': [15], 'RoomTemp': [22], 'RoomHumidity': [48], 'FlowRate': [9],
    'ColdWaterTemp': [16], 'Activity_Dishwashing': [0], 'Activity_Hand Washing': [0], 'Activity_Laundry': [0], 'Activity_Shower': [1],
    'TimeOfDay_Afternoon': [1], 'TimeOfDay_Evening': [0], 'TimeOfDay_Morning': [0],
    'Season_Autumn': [1], 'Season_Spring': [0], 'Season_Summer': [0], 'Season_Winter': [0]
}

example_df = pd.DataFrame(new_data)

# Predict the desired temperature
predicted_temp = model.predict(example_df)
print(f'Predicted Desired Temperature: {predicted_temp[0]}')


model_filename = 'model.pkl'
joblib.dump(model, model_filename)

Mean Squared Error: 0.0
R2 Score: 1.0
Predicted Desired Temperature: 39.0


['model.pkl']